In [7]:
from urllib.parse import quote
from urllib import request
import codecs, jieba
import re, html, time
from os import listdir
from os.path import isfile, join
import random

jieba.add_word('龐傍燮謝', freq=10, tag='xx')

class GoogleCrawler(object):
    def __init__(self, qno, wlist, qidx, anslist):
        self.qno = qno
        self.wlist = wlist
        self.iqlist = qidx
        self.anslist = anslist
        self.qnum = 2
        self.rand_query = []
        self.set_rand_query()
    
    def add_rand_query(self, rwlist = None, qidx = None):
        if rwlist == None or qidx == None:
            rwlist = self.wlist
            qidx = self.iqlist
            rwlen = len(rwlist)
            
            for i in range(self.qnum):
                iq = random.choice(qidx)
                if iq > 0:
                    ihead = random.choice(range(0, iq))
                else:
                    ihead = 0
                if iq + 1 < rwlen:
                    iend = random.choice(range(iq + 1, rwlen))
                else:
                    iend = rwlen- 1
                self.rand_query.append(''.join(rwlist[ihead : iend]))
                
    def set_rand_query(self):
        self.rand_query = []
        self.add_rand_query()
    
    def get_rand_query(self):
        return self.rand_query
    
    def print_rand_query(self):
        for q in self.rand_query:
            print(q)
    
    def crawl(self, query):
        raw_html = self.google_crawl(query).lower()
        clean_html = self.clean_html(raw_html)
        return clean_html
    
    def multitask_craw(self):
        pass
    
    def fast_search(self):
        html_pool = ''
        for q in self.rand_query:
            html_pool += self.crawl(q)
            time.sleep(0.5)
        for x in self.anslist:
            if html_pool.find(x) > 0:
                return (self.anslist.index(x), x)
        
    def google_crawl(self, query = None):
        if query == None: query = self.short_query
        self.link = "https://www.google.com/search?q=" + quote(query)  + '&ie=utf8&oe=utf8' # + '&lr=lang_zh-TW'
        req = request.Request(self.link, headers = {'User-Agent' : "Chrome Browser"})
        raw = html.unescape(request.urlopen(req).read().decode('utf-8'))
        return raw
    
    def clean_html(self, raw_html = None):
        if raw_html == None: raw_html = self.google_crawl(self.short_query)
        clean_html = re.sub(re.compile(r'(<br?>)|(</br?>)|\n|\r|\s'), '', raw_html)
        return clean_html

In [11]:
# import random
# question = ['*', '水利局', '副局長', '彭紹博', '表示','，', '該工程','是',
#             '經濟部', '水利署', '所', '建', '，', '但', '因', '工程', '已經', '完工','，',
#             '市府', '針對', '此','狀況','，', '正', '積極', '尋求','補救','，','希望','在',
#             '兩岸', '闢建', '公園', '，', '*', '但', '因', '地價', '太高', '，', '還在', '努力中']
# q_index = [0,35]

# crawler = GoogleCrawler(question, q_index)
# crawler.print_rand_query()
sample = '''\
[1] 熱水池可以容納二大二小沒問題，果然到︽⊙＿⊙︽泡無色無味的溫泉很舒服，泡的身子暖呼呼，\
回民宿後果然很好睡呀 ### a: 聚會, b: 礁溪, c: 貝殼, d: 高昂, e: 開機 [END]'''

jieba.add_word('龐傍燮謝', freq=10, tag='xx')

def simple_preprocess(sample):
    no, content, a, b, c, d, e = re.findall(r'\[(\d+)\](.*)### a:(.*), b:(.*), c:(.*), d:(.*), e:(.*)\[END\]', sample)[0]
    options = [a.strip(), b.strip(), c.strip(), d.strip(), e.strip()]
    content = re.sub('(")(.?)', r'\2', content.strip())
    content = content.strip().replace('︽⊙＿⊙︽', '龐傍燮謝')
    wlist = list(jieba.cut(content))
    qidx = []
    i = 0
    for w in wlist:
        if w == '龐傍燮謝':
            wlist[i] = '*'
            qidx.append(i)
        i += 1
    return (no, wlist, qidx, options)

path = 'question_samples/'
filename = '2016-07-31-15-02-02.txt'
no = 4
def get_sample(file_name, no_sample):
    with codecs.open(file_name, 'r', 'utf-8') as f:
        for line in f:
            for (no, content, a, b, c, d, e) in re.findall(r'\[(\d+)\](.*)### a:(.*), b:(.*), c:(.*), d:(.*), e:(.*)\[END\]', line):
                if int(no) == int(no_sample):
                    return line

no, wlist, qidx, anslist = simple_preprocess(get_sample(path + filename, no))
print(no)
print(wlist)
print(qidx)
crawler = GoogleCrawler(no, wlist, qidx, anslist)
crawler.fast_search()

4
['因為', '地處', '*', '的', '高點', '-', '-', '號稱', '"', '全', '*', '最高', '的', '溫泉', '"', ',', '又', '有', '"', '*', '101', '"', '的', '別']
[2, 10, 19]


(0, '烏來')

In [ ]:
path = 'question_samples/'
sample_files = [f for f in listdir(path) if isfile(join(path, f))]
i, j = 0, 0
for file in sample_files:
    with codecs.open(path + file, encoding='utf-8') as f:
        for line in f:
            if line.find('︽⊙＿⊙︽') > 0:
                no, wlist, qidx, anslist = simple_preprocess(line)
                crawler = GoogleCrawler(no, wlist, qidx, anslist)
                try:
                    ans = crawler.fast_search()
                    if ans == None:
                        print('[failure]{}:{}'.format(file, line[:3]))
                        j += 1
                except:
                    print('[exception]:{}:{}'.format(file, line[:3]))
                    j += 1
                i += 1
                if i % 100 == 1: print(1. - float(j) / float(i))
                time.sleep(0.5)
print(1. - float(j) / float(i))

1.0


In [31]:
line
# print(crawler.wlist)
# print(crawler.rand_query)

'[1] 這裡之前gisele跟hannah都跟我大力推薦, 因為這個spa用的產品全部都是來自約旦死海的︽⊙＿⊙ \t### a: 保養品, b: 陰雨, c: 柿餅, d: 冬粉, e: 問度 [END]\n'